In [5]:
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as pp
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# 11.1

In [21]:
# vertices
V = set(range(10))
# N[v]: neighbors of v
N = [
    {2,4,7}, 
    {2,3,4,6},
    {0,1,3,6,7,8},
    {1,2},
    {0,1,5,6},
    {4,9},
    {1,2,4,7,8},
    {0,2,6,8},
    {2,6,7},
    {5}
]

letter = ['a','b','c','d','e','f','g','h','i','j']

# Bron Kerbosch Alg.
def find_all_maximal_cliques(R, P, X):
    if not P and not X:
        clique = [letter[v] for v in R]
        print("".join(sorted(clique)))

    for v in P:
        find_all_maximal_cliques(R | {v}, P & N[v], X & N[v])
        P = P - {v}
        X = X | {v}

    return

print "exercise 11.1"
find_all_maximal_cliques(set(), V, set())

print "\nmoral graph"
V = set(range(5))
N = [{1,2}, {0,2}, {0,1,3,4}, {2}, {2}]
find_all_maximal_cliques(set(), V, set())

exercise 11.1
ach
ae
bcd
bcg
beg
cghi
ef
fj

moral graph
abc
cd
ce


# 11.2

# 11.3

In [19]:
# the graph in (a)
def p(b=[0,1],e=[0,1],a=[0,1],r=[0,1]):
    if isinstance(b, int): b = [b]
    if isinstance(e, int): e = [e]
    if isinstance(a, int): a = [a]
    if isinstance(r, int): r = [r]
   
    # variable syntax: p_...x := p(x|...)
    p_b = [0.99, 0.01]
    p_e = [1.-10e-6, 10e-6]
    p_re = [[1.0, 0.0], [0.0, 1.0]]
    p_bea = [[[0.999, 0.001], [0.59, 0.41]], [[0.05, 0.95], [0.02, 0.98]]]
 
    p_sum = 0.0
    for bi in b:
        for ei in e:
            for ai in a:
                for ri in r:
                    p_sum += p_b[bi] * p_e[ei] * p_re[ri][ei] * p_bea[bi][ei][ai]

    return p_sum

def example():
    print "p(b=1) =", p(b=1)
    print "p(b=1|e=1) =", p(b=1,e=1) / p(e=1)
    print
    
    for b in [1,0]:
        for e in [1,0]:
            for a in [1,0]:
                p_ab = p(b=b,a=a) / p(a=a)
                p_eab = p(b=b,e=e,a=a) / p(e=e,a=a)
                sign = "< " if p_eab < p_ab else ">="
                print "p(b=%i|e=%i,a=%i)" % (b,e,a), sign, "p(b=%i|a=%i)" % (b,a)

    print
    for a in [0,1]: print "p(a=%i) == %f" % (a, p(a=a))
    for b in [0,1]: print "p(b=%i) == %f" % (b, p(b=b))
    for e in [0,1]: print "p(e=%i) == %f" % (e, p(e=e))
    for r in [0,1]: print "p(r=%i) == %f" % (r, p(r=r))
    return
example()

p(b=1) = 0.01
p(b=1|e=1) = 0.01

p(b=1|e=1,a=1) <  p(b=1|a=1)
p(b=1|e=1,a=0) <  p(b=1|a=0)
p(b=1|e=0,a=1) >= p(b=1|a=1)
p(b=1|e=0,a=0) >= p(b=1|a=0)
p(b=0|e=1,a=1) >= p(b=0|a=1)
p(b=0|e=1,a=0) >= p(b=0|a=0)
p(b=0|e=0,a=1) <  p(b=0|a=1)
p(b=0|e=0,a=0) <  p(b=0|a=0)

p(a=0) == 0.989506
p(a=1) == 0.010494
p(b=0) == 0.990000
p(b=1) == 0.010000
p(e=0) == 0.999990
p(e=1) == 0.000010
p(r=0) == 0.999990
p(r=1) == 0.000010


In [17]:
from bayesian.bbn import build_bbn
from bayesian.utils import make_key

def f_burglary(burglary):
    if burglary:
        return 0.01
    else:
        return 0.99

def f_earthquake(earthquake):
    if earthquake:
        return 1e-6
    else:
        return 1.-1e-6

def f_alarm(burglary, earthquake, alarm):
    table = dict()
    table['fff'] = 0.999
    table['fft'] = 0.001
    table['ftf'] = 0.59
    table['ftt'] = 0.41
    table['tff'] = 0.05
    table['tft'] = 0.95
    table['ttf'] = 0.02
    table['ttt'] = 0.98
    return table[make_key(burglary, earthquake, alarm)]

def f_radio(earthquake, radio):
    table = dict()
    table['ff'] = 1.0
    table['ft'] = 0.0
    table['tf'] = 0.0
    table['tt'] = 1.0
    return table[make_key(earthquake, radio)]

if __name__ == '__main__':
    g = build_bbn(
        f_burglary,
        f_earthquake,
        f_alarm,
        f_radio)
    g.q()

+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.989510 |
| alarm      | True  | 0.010490 |
| burglary   | False | 0.990000 |
| burglary   | True  | 0.010000 |
| earthquake | False | 0.999999 |
| earthquake | True  | 0.000001 |
| radio      | False | 0.999999 |
| radio      | True  | 0.000001 |
+------------+-------+----------+
